In [15]:
# !git clone https://github.com/bricerauby/MRI_Scoring.git
# %cd /content/MRI_Scoring
# from google.colab import drive
# drive.mount('/content/drive')
# !pip3 install -r requirements.txt
# assert os.environ['COLAB_TPU_ADDR']
# TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']

Clonage dans 'MRI_Scoring'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 37 (delta 9), reused 30 (delta 5), pack-reused 0
Dépaquetage des objets: 100% (37/37), fait.
[Errno 2] No such file or directory: '/content/MRI_Scoring'
/home/brice/Documents/3A/kaggle/challenge_JFR


ModuleNotFoundError: No module named 'google.colab'

In [21]:
%load_ext autoreload
%autoreload 2
import src.utils
import src.model
import numpy as np 
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
def parse_fn(example):
    "Parse TFExample records and perform simple data augmentation."
    example_fmt = {
        'age': tf.io.FixedLenFeature((), tf.int64, -1),
        'Sequence_id': tf.io.FixedLenFeature((), tf.string, ""),
        'EDSS': tf.io.FixedLenFeature((), tf.float32, 0.),
        'examination_date': tf.io.FixedLenFeature((), tf.string, ""),
        'serie_description': tf.io.FixedLenFeature((), tf.string, ""),
        'h': tf.io.FixedLenFeature((), tf.int64, 0),
        'w': tf.io.FixedLenFeature((), tf.int64, 0),
        'd': tf.io.FixedLenFeature((), tf.int64, 0),
        'r_h': tf.io.FixedLenFeature((), tf.float32, 0),
        'r_w': tf.io.FixedLenFeature((), tf.float32, 0),
        'r_d': tf.io.FixedLenFeature((), tf.float32, 0),
        'image_raw':  tf.io.FixedLenFeature((), tf.string, "")
    }
    parsed = tf.parse_single_example(example, example_fmt)
    image = tf.decode_raw(parsed['image_raw'],out_type=tf.float64)
    image = tf.reshape(image, shape=(parsed['h'],
                                     parsed['w'],
                                     parsed['d'],
                                     1) )

    return image, parsed["EDSS"]

def train_input_fn(batch_size=8, buffer_size=8*1024*1024, prefetch_buffer_size=2, num_workers=8):
    files = tf.data.Dataset.list_files("data/tf_records/train_set_*.tfrecords")
    dataset = files.apply(tf.contrib.data.parallel_interleave(tf.data.TFRecordDataset, 
                                                              cycle_length=num_workers))
    dataset = dataset.shuffle(buffer_size=buffer_size)
    dataset = dataset.repeat()
    dataset = dataset.map(map_func=parse_fn, num_parallel_calls=num_workers)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=prefetch_buffer_size)
    return dataset




In [24]:
dataset = train_input_fn()
model = src.model.dumb_model()
model.compile(
  optimizer='adam',
  loss= 'categorical_crossentropy',
  metrics=['accuracy'])

In [ ]:
model.fit(dataset, epochs=1, steps_per_epoch=100)